In [ ]:
%pip install --upgrade pip
%pip install --upgrade transformers datasets[audio] accelerate
%pip install ipywidgets
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
%pip install fastapi uvicorn python-multipart

In [ ]:
import torch
import uvicorn
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from fastapi import FastAPI, UploadFile, File
from io import BytesIO

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

print(device)
print(torch_dtype)

In [ ]:
model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

In [ ]:
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True
)

In [ ]:
file_path = "test3.wav"

with open(file_path, "rb") as f:
  audio_bytes = f.read()

import numpy as np
import soundfile as sf

audio, _ = sf.read(BytesIO(audio_bytes))
result = pipe(audio)

print(result)

In [ ]:
app = FastAPI()

@app.post("/transcribe/")
async def transcribe(file: UploadFile = File(...)):
  audio_bytes = await file.read()
  audio = BytesIO(audio_bytes)
  result = pipe(audio)
  return result

if __name__ == "__main__":
  uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
import requests

url = "http://127.0.0.1:8000/transcribe/"
file_path = "test.wav"

with open(file_path, "rb") as f:
  files = {"file": f}
  response = requests.post(url, files=files)

print(response.json())